In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
from depsurf import (
    OUTPUT_PATH,
    VERSION_LAST,
    FuncGroups,
    InlineType,
    Version,
    SymbolTable,
)


def count_inline(v: Version):
    groups = FuncGroups.from_dump(v.dwarf_funcs_path)
    results = {t: FuncGroups() for t in InlineType}
    symtab = SymbolTable.from_dump(v.symtab_path)

    for name, group in groups.iter_groups():
        in_symtab = name in symtab.func_sym_groups
        inline_type = group.get_inline_type(in_symtab=in_symtab)
        results[inline_type].add_group(name, group)

    for t, group in results.items():
        group.save_funcs(OUTPUT_PATH / "inline" / v.name / f"{t.name.lower()}.txt")
    
    total = groups.num_groups

    return {
        **{t: group.num_groups for t, group in results.items()},
        **{f"{t}%" : 100 * group.num_groups / total for t, group in results.items()},
    }


# count_inline(VERSION_LAST)

In [5]:
from depsurf import Versions
from depsurf.output import save_df

df = Versions.apply(count_inline, groups=[Versions.REGULAR, Versions.ARCH, Versions.FLAVOR])

save_df(df, "inline")

[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.4.0-21-generic-amd64.jsonl


[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.4.0-21-generic-amd64.jsonl
[        group.py:75 ] WARNING: xen_pvh_early_cpu_init has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.py:75 ] WARNING: __builtin_strlen has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.py:75 ] WARNING: relocate_kernel has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.py:75 ] WARNING: __builtin_strchr has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.py:75 ] WARNING: swsusp_arch_suspend has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.py:75 ] WARNING: __builtin_strcpy has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.p

Fully inlined  Not inlined  Partially inlined  Fully inlined%  Not inlined%  Partially inlined%
Regular 4.4.0-21-generic-amd64           19900.0      31197.0             6306.0       34.667178     54.347334           10.985489
        4.8.0-22-generic-amd64           20945.0      32826.0             6610.0       34.688064     54.364784           10.947152
        4.10.0-19-generic-amd64          21649.0      33643.0             6744.0       34.897479     54.231414           10.871107
        4.13.0-16-generic-amd64          22717.0      34636.0             7445.0       35.058181     53.452267           11.489552
        4.15.0-20-generic-amd64          23382.0      35977.0             7712.0       34.861565     53.640172           11.498263
        4.18.0-10-generic-amd64          24765.0      38703.0             7181.0       35.053575     54.782092           10.164334
        5.0.0-13-generic-amd64           25599.0      39450.0             7259.0       35.402722     54.558278           10.039000
        5.3.0-18-generic-amd64           26520.0      41203.0             7350.0       35.325616     54.883913            9.790471
        5.4.0-26-generic-amd64           27224.0      42040.0             7541.0       35.445609     54.736020            9.818371
        5.8.0-25-generic-amd64           26090.0      45295.0             8291.0       32.745118     56.848988           10.405894
        5.11.0-16-generic-amd64          27074.0      48400.0             8416.0       32.273215     57.694600           10.032185
        5.13.0-19-generic-amd64          29570.0      47352.0             8270.0       34.709832     55.582684            9.707484
        5.15.0-25-generic-amd64          30203.0      48216.0             8186.0       34.874430     55.673460            9.452110
        5.19.0-21-generic-amd64          31313.0      50330.0             7957.0       34.947545     56.171875            8.880580
        6.2.0-20-generic-amd64           32516.0      51429.0             8103.0       35.325048     55.871936            8.803016
        6.5.0-9-generic-amd64            33495.0      53587.0             8461.0       35.057513     56.086788            8.855698
        6.8.0-22-generic-amd64           34907.0      55827.0             8912.0       35.031010     56.025330            8.943661
Arch    5.4.0-26-generic-arm64           27547.0      43427.0             7676.0       35.024793     55.215512            9.759695
        5.4.0-26-generic-armhf           26198.0      43148.0             6671.0       34.463344     56.760988            8.775669
        5.4.0-26-generic-ppc64el         24310.0      37962.0             6146.0       35.531585     55.485399            8.983016
        5.4.0-26-generic-s390x           20662.0      28180.0             6169.0       37.559761     51.226118           11.214121
Flavor  5.4.0-26-lowlatency-amd64        27248.0      42077.0             7526.0       35.455622     54.751402            9.792976
        5.4.0-1009-aws-amd64             26722.0      40737.0             7311.0       35.738933     54.483081            9.777986
        5.4.0-1009-gcp-amd64             27316.0      42168.0             7552.0       35.458747     54.738045            9.803209
        5.4.0-1009-oracle-amd64          28093.0      43170.0             7743.0       35.558059     54.641420            9.800521
        5.4.0-1010-azure-amd64           26304.0      39861.0             7093.0       35.905976     54.411805            9.682219